# Week 2

## Word representation

* Problems with the standard 1-hot presentation of words: model finds it hard to learn relationships between words (king is to queen as man is to woman):
  * Inner product between any 2 one-hot vectors is 0.
  * Euclidean distance between any pair of vectors is the same.
* Would be better to learn a set of features for each words that help to categorise it (eg gender, royal-ness, age, fruit-ness etc).
* Embedding basically learns relevant features (usually not easy to interpret), generating a 300 (or whatever) dimensions vector for each word.

## Using word embeddings

* Named entity recogition example: what happens if you come across a set of words that your model hasn't learning in the standard RNN approach? Enter embedding.
* Embedding can be trained on much larger corpuses of text (1B - 100B words is not uncommon).
* Embedding allows for transfer learning: models using pretrained embeddings can be trained on smaller corpuses (à la ImageNet models in the image classificatino word).
  * Embeddings can be finetuned with new data (if dataset if big enough).
* Embeddings have been useful for: entity recognition, text summarisation, co-reference resolution, parsing.
* Less useful for: language modeling, machine translation.

### Properties of word embeddings

* Word embeddings can help with "analogy reasoning".
  * Answering the question: "Man is to woman as king is to ___ ?"
* With a 4 dimension embedding for man and woman, with a feature for gender, royalness, age and foodness, you could subtract them from each other:

    ```
    e_man - e_woman = [-2, 0, 0, 0]
    ```
    
  If you did the same from king and queen, you may end up with a similar result:
   
    ```
    e_king - e_queen = [-2, 0, 0, 0]
    ```
   
  * What it captures is that the main different between the two word sets is the gender.
  * Ideas first published in paper [Linguistic Regularities in Continuous Space Word Representations](https://www.aclweb.org/anthology/N13-1090).
* Formally, you'd aim to find the $e_{w} that satisfies that expression:
   
   $e_{man} - e_{woman} \approx e_{king} - e_{w}$
   
   * Find word $e_{w}$ that maximises this the similarity expression (ie has the highest degree of similiarity): $\text{sim}(e_{w}, e_{king} - e_{man} + e_{woman})$
   
* Similarity function: cosine similarity:
  * $sim(u, v) = \frac{u^{t}v}{||u||_2 ||v||_1}$
    * Without the denominator, it's basically the inner product: if they are similar, product will be large.
    * With the denominator, normalises result between -1 and 1. Very distant = -1, very similar = 1.
  * Could also use squared distance: $||u-v||^2$ (since it's a measure of dissimilarity, you should take the negative to maximise.

### Embedding matrix

